# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import logspace, concatenate
from misvm import SIL, MICA, MISVM, miSVM, NSK, stMIL, sMIL, sbMIL, STK
from sklearn.preprocessing import MinMaxScaler
from toolbox.classification.common import Data,  IO, Tools
from toolbox.classification.parameters import Settings
from toolbox.models.models import CustomMIL
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import ViewsTools, Views
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type = 'Full'
extractor = 'Frequency'
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs_instances = IO.load(f'Features.pickle')
inputs_bags = IO.load(f'Features.pickle')

In [ ]:
inputs_instances = inputs_instances[(inputs_instances['Binary_Diagnosis'] == 'Benign') |(inputs_instances['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

In [ ]:
inputs_bags = inputs_bags[(inputs_bags['Binary_Diagnosis'] == 'Benign') |(inputs_bags['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

# Model evaluation

## Models

In [ ]:
models = []
models.append(('MISVM',
               CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False)),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))
models.append(('Scale_MISVM',
               CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False), data_preparation=MinMaxScaler()),
               {'estimator__C': logspace(-2, 3, 6).tolist()}))

In [ ]:
weak_file = f'Weak_Frequency_Instances.pickle'

In [ ]:
# Simple labels
all_image = [True] * len(inputs_bags.index)
single_image = inputs_bags['ID_Image'] == '0M'
Data.build_bags(inputs_bags, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
inputs_bags = inputs_bags[single_image].reset_index()

In [ ]:
# for model in models:
#     model_name, model_kernel,model_params = model
#     # MIL evaluation
#     Tools.evaluate(inputs_bags, {'datum': extractor, 'label_encode': 'LesionEncode'}, model_kernel, model_name, distribution=model_params, instance=True)

# for model in models:
#     model_name, model_kernel,model_params = model
#     inputs_instances[f'{model_name}_Prediction'] = concatenate(inputs_bags[f'{model_name}_Prediction'])
#     inputs_instances[f'{model_name}_Probability'] = [x for x in concatenate(inputs_bags[f'{model_name}_Probability'])]

# # Save
# IO.save(inputs_instances, weak_file)

## Scores and ROC

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs_inst = IO.load(weak_file)
inputs_inst = inputs_inst[inputs_inst['Label']!='Unknown'].reset_index(drop=True)

malignant_encoder = OrderedEncoder().fit(['Rest', 'Malignant'])
Tools.transform(inputs_inst, {'datum': 'Malignant'}, malignant_encoder, 'MalignantEncode')

# ROC Curve
ViewsTools.plot_size((8,8))
    
for model in models:  
    # Decompose model
    model_name, model_process, model_params = model        
    
    name = model_name
    # Label
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs_inst, {'label_encode': 'MalignantEncode', 'eval': name}, malignant_encoder)],
                                                                        title=[f'Test - {name}'])))

In [ ]:
inputs_inst = IO.load(weak_file)
inputs_inst = inputs_inst[inputs_inst['Label']!='Unknown'].reset_index(drop=True)

malignant_encoder = OrderedEncoder().fit(['Rest', 'Malignant'])
Tools.transform(inputs_inst, {'datum': 'Malignant'}, malignant_encoder, 'MalignantEncode')

# ROC Curve
ViewsTools.plot_size((8,8))
    
for model in models:  
    # Decompose model
    model_name, model_process, model_params = model        
    
    name = model_name
    # Label
    inputs_lm = inputs_inst[(inputs_inst['Diagnosis']=='LM/LMM')]
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs_lm, {'label_encode': 'MalignantEncode', 'eval': name}, malignant_encoder)],
                                                                        title=[f'Test LM- {name}'])))